In [273]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [274]:
resp = requests.get('https://data.ct.gov/resource/rf3k-f8fg.json')
txt = resp.json()

In [275]:
dat_orig = pd.DataFrame(txt)

In [276]:
dat = dat_orig[dat_orig['state']=='CONNECTICUT'].copy()

dat = dat[['date', 'cases', 'deaths', 'cases_age0_9']]

cols = dat.columns.drop('date')

dat[cols] = dat[cols].apply(pd.to_numeric, errors='coerce')

dat['date'] = dat['date'].apply(pd.to_datetime)

In [277]:
sort_dat = dat.sort_values(by=['date'], inplace=False, ascending=True).copy()

In [278]:
sort_dat['daily_new_cases'] = sort_dat['cases'].diff()
sort_dat['daily_new_deaths'] = sort_dat['deaths'].diff()
sort_dat['daily_new_child_cases'] = sort_dat['cases_age0_9'].diff()

In [279]:
sort_dat['new_cases_roll3'] = sort_dat['daily_new_cases'].rolling(3).mean().round()
sort_dat['new_cases_roll7'] = sort_dat['daily_new_cases'].rolling(7).mean().round()
sort_dat['new__child_cases_roll3'] = sort_dat['daily_new_child_cases'].rolling(3).mean().round()
sort_dat['new__child_cases_roll7'] = sort_dat['daily_new_child_cases'].rolling(7).mean().round()
sort_dat['new_deaths_roll3'] = sort_dat['daily_new_deaths'].rolling(3).mean().round()
sort_dat['new_deaths_roll7'] = sort_dat['daily_new_deaths'].rolling(7).mean().round()

In [ ]:
plt.figure(num = 20, figsize=(15, 10))
plt.plot(sort_dat['date'], sort_dat['new_cases_roll7'])
# plt.plot(sort_dat['date'], sort_dat['new_cases_roll3'])
# plt.plot(sort_dat['date'], sort_dat['daily_new_cases'])
# plt.plot(sort_dat['date'], sort_dat['new__child_cases_roll7'])
# plt.plot(sort_dat['date'], sort_dat['new_deaths_roll7'])

plt.show()

In [283]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


In [376]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/gmail.compose',
         'https://www.googleapis.com/auth/gmail.send']

# Gmail scopes: https://developers.google.com/identity/protocols/oauth2/scopes#gmail
# Sheets scopes: https://developers.google.com/identity/protocols/oauth2/scopes#sheets

def build_serv():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    sheets = build('sheets', 'v4', credentials=creds)
    mail = build('gmail', 'v1', credentials=creds)
    service = {'sheets': sheets,
              'mail': mail}
    return(service)


In [377]:
service = build_serv()
mail_service = service['mail']
sheets_service = service['sheets']

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=485047975673-9tt2qa6bgrcme7olhn657o5k9svbo6vu.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A63728%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.compose+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=K8m3zqD4MBt5YcHNTBS6RvnfWokAgo&access_type=offline


In [381]:
def read_data(service, spreadsheet, sheet_range):
    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet, 
                                range=sheet_range).execute()
    values = result.get('values', [])
    return(values)

In [382]:
def send_data(service, dat, date_col, spreadsheet_id, sheet_range):
    send_dat = dat.copy()
    send_dat[date_col] = send_dat[date_col].dt.strftime('%Y-%m-%d')
    send_dat.fillna('', inplace=True)
    
    # How the input data should be interpreted.
    value_input_option = 'USER_ENTERED'  # USER_ENTERED to mimic Sheets entry, RAW to place data as-is

    # How the input data should be inserted.
    insert_data_option = 'OVERWRITE'  # not being used at the moment, function overwrites by default

    dfHeaders = send_dat.columns.values.tolist()
    dfHeadersArray = [dfHeaders]
    dfData = send_dat.values.tolist()

    value_range_body = {
            "majorDimension": "ROWS",
            "values": dfHeadersArray + dfData
        }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, 
                                                     range=sheet_range, 
#                                                      insertDataOption=insert_data_option,
                                                     valueInputOption=value_input_option, 
                                                     body=value_range_body)
    response = request.execute()

    return(response)





In [383]:
# Update Google Sheet with latest data
send_data(sheets_service, sort_dat, 'date', '1h8PgeXtEDaHQCmsFIcqDA3Hm_kVFRrR-jcEfVPfMwAI', 'data')

{'spreadsheetId': '1h8PgeXtEDaHQCmsFIcqDA3Hm_kVFRrR-jcEfVPfMwAI',
 'updatedRange': 'data!A1:M53',
 'updatedRows': 53,
 'updatedColumns': 13,
 'updatedCells': 689}

In [384]:
# Load alerts from Google Sheet
def pull_alerts(service, spreadsheet_id, sheet_range):
    alerts = read_data(service, spreadsheet_id, sheet_range)
    alerts = pd.DataFrame(alerts)
    new_header = alerts.iloc[0] #grab the first row for the header
    alerts = alerts[1:] #take the data less the header row
    alerts.columns = new_header #set the header row as the df header
    return(alerts)

In [395]:
alerts = pull_alerts(sheets_service, '1h8PgeXtEDaHQCmsFIcqDA3Hm_kVFRrR-jcEfVPfMwAI', 'alerts')

In [400]:
# alerts

In [390]:
latest_day = sort_dat.iloc[len(sort_dat)-1]

carriers = {'verizon':'@vtext.com',
            'xfinity':'@vtext.com', 
            'google_fi':'@msg.fi.google.com', 
            'tmobile':'@tmomail.net',
           'virgin':'@vmobl.com', 
           'att':'@txt.att.net',
           'sprint':'@messaging.sprintpcs.com',
           'boost':'@sms.myboostmobile.com',
           'metro':'@mymetropcs.com',
           'us_cellular':'@email.uscc.net',
           'cricket':'@mms.cricketwireless.net',
           'republic':'@text.republicwireless.com',
           'straight_talk':'@vtext.com'}



In [391]:
from apiclient import errors
from httplib2 import Http
from email.mime.text import MIMEText
import base64
from google.oauth2 import service_account

def create_message(sender, to, subject, message_text):
    """Create a message for an email.
    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.
    Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode()).decode()}

def send_message(service, user_id, message):
    """Send an email message.
    Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.
    Returns:
    Sent Message.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                   .execute())
        print('Message Id: %s' % message['id'])
        return message
    except errors.HttpError as error:
        print('An error occurred: %s' % error)

In [399]:
EMAIL_FROM = 'jcdrummr@gmail.com'

for row in alerts.itertuples():
    if latest_day[row.metric] <= float(row.threshold):
        EMAIL_TO = row.phone + carriers[row.carrier]
        EMAIL_SUBJECT = 'CT COVID-19 Alert System'
        EMAIL_CONTENT = row.metric + ' is now at ' + \
                        str(latest_day[row.metric]) + ', below your alert threshold of ' + \
                        str(row.threshold)
        # Call the Gmail API
        message = create_message(EMAIL_FROM, EMAIL_TO, EMAIL_SUBJECT, EMAIL_CONTENT)
        sent = send_message(mail_service,'me', message)

Message Id: 1721ac042e6f934d
Message Id: 1721ac0421010488
